In [1]:
import re
import pandas as pd
from itertools import product

def results_text_to_clipboard_df(text: str):
    """
    Parse lines like:
      [train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.29588704978190755
    Build a DF with columns: event, 5_1, 5_2, 5_3, 10_1, ... 50_3
    Copy DF to clipboard for easy paste into Sheets.
    """
    # event, lb size, set number, score
    rx = re.compile(
        r"\[train\[(.+?)_lb(\d+)_set(\d+)\]\]\s*macro-F1\s*=\s*([0-9]*\.?[0-9]+)",
        flags=re.IGNORECASE
    )

    rows = []
    for m in rx.finditer(text):
        event, lb, setnum, score = m.group(1), m.group(2), m.group(3), m.group(4)
        rows.append((event, int(lb), int(setnum), float(score)))

    if not rows:
        print("No matches found. Check the text format or regex.")
        return None

    # Build wide table
    df_long = pd.DataFrame(rows, columns=["event", "lb", "set", "score"])
    # Keep only the requested lb/set grid
    wanted_lbs  = [5, 10, 25, 50]
    wanted_sets = [1, 2, 3]
    df_long = df_long[df_long["lb"].isin(wanted_lbs) & df_long["set"].isin(wanted_sets)]

    # Pivot to event rows, columns like "5_1"
    df_wide = df_long.assign(col=lambda d: d["lb"].astype(str) + "_" + d["set"].astype(str)) \
                     .pivot_table(index="event", columns="col", values="score", aggfunc="first")

    # Ensure full cartesian column order exists
    all_cols = [f"{lb}_{s}" for lb, s in product(wanted_lbs, wanted_sets)]
    df_wide = df_wide.reindex(columns=all_cols)

    # Put 'event' as an actual column (since you asked for it in the output)
    df_out = df_wide.reset_index()

    # Copy to clipboard
    # df_out.to_clipboard(index=False)
    print("✅ Copied to clipboard. Preview:")
    print(df_out)
    return df_out

# ---- Example usage ----
text = """[train[california_wildfires_2018_lb5_set1]] macro-F1 = 0.09034619051696481
[train[california_wildfires_2018_lb5_set2]] macro-F1 = 0.36001293328096184
[train[california_wildfires_2018_lb5_set3]] macro-F1 = 0.3700721745215991
[train[hurricane_irma_2017_lb5_set1]] macro-F1 = 0.15136262483480317
[train[hurricane_irma_2017_lb5_set2]] macro-F1 = 0.4125014700113703
[train[hurricane_irma_2017_lb5_set3]] macro-F1 = 0.22136114542526866
[train[cyclone_idai_2019_lb5_set1]] macro-F1 = 0.1307519745246755
[train[cyclone_idai_2019_lb5_set2]] macro-F1 = 0.21190792002170672
[train[cyclone_idai_2019_lb5_set3]] macro-F1 = 0.25865909330401304
[train[hurricane_florence_2018_lb5_set1]] macro-F1 = 0.32722289510386726
[train[hurricane_florence_2018_lb5_set2]] macro-F1 = 0.4581250706988753
[train[hurricane_florence_2018_lb5_set3]] macro-F1 = 0.39723151812127483
[train[hurricane_harvey_2017_lb5_set1]] macro-F1 = 0.19182169000424473
[train[hurricane_harvey_2017_lb5_set2]] macro-F1 = 0.13999572681409522
[train[hurricane_harvey_2017_lb5_set3]] macro-F1 = 0.3186680522378848
[train[kerala_floods_2018_lb5_set1]] macro-F1 = 0.2744593935172427
[train[kerala_floods_2018_lb5_set2]] macro-F1 = 0.3437907588874751
[train[kerala_floods_2018_lb5_set3]] macro-F1 = 0.21656596601992034
[train[hurricane_dorian_2019_lb5_set1]] macro-F1 = 0.31780949925881213
[train[hurricane_dorian_2019_lb5_set2]] macro-F1 = 0.32557652317559355
[train[hurricane_dorian_2019_lb5_set3]] macro-F1 = 0.43085295872969326
[train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.38794002330238203
[train[canada_wildfires_2016_lb5_set2]] macro-F1 = 0.2605002309168516
[train[canada_wildfires_2016_lb5_set3]] macro-F1 = 0.4137894133308759
[train[hurricane_maria_2017_lb5_set1]] macro-F1 = 0.2712945209227838
[train[hurricane_maria_2017_lb5_set2]] macro-F1 = 0.3787898645932697
[train[hurricane_maria_2017_lb5_set3]] macro-F1 = 0.45279678067139095
[train[kaikoura_earthquake_2016_lb5_set1]] macro-F1 = 0.4675387896355855
[train[kaikoura_earthquake_2016_lb5_set2]] macro-F1 = 0.220182264814113
[train[kaikoura_earthquake_2016_lb5_set3]] macro-F1 = 0.5090410356034822
[train[california_wildfires_2018_lb10_set1]] macro-F1 = 0.5132184268642916
[train[california_wildfires_2018_lb10_set2]] macro-F1 = 0.4793580331132027
[train[california_wildfires_2018_lb10_set3]] macro-F1 = 0.449941057961897
[train[hurricane_irma_2017_lb10_set1]] macro-F1 = 0.5416947946007155
[train[hurricane_irma_2017_lb10_set2]] macro-F1 = 0.44806941641043196
[train[hurricane_irma_2017_lb10_set3]] macro-F1 = 0.5058020260294066
[train[cyclone_idai_2019_lb10_set1]] macro-F1 = 0.4323117322849922
[train[cyclone_idai_2019_lb10_set2]] macro-F1 = 0.3051154549972181
[train[cyclone_idai_2019_lb10_set3]] macro-F1 = 0.3332746413000146
[train[hurricane_florence_2018_lb10_set1]] macro-F1 = 0.508538388482164
[train[hurricane_florence_2018_lb10_set2]] macro-F1 = 0.5907146670053172
[train[hurricane_florence_2018_lb10_set3]] macro-F1 = 0.5046066712831977
[train[hurricane_harvey_2017_lb10_set1]] macro-F1 = 0.563430652588792
[train[hurricane_harvey_2017_lb10_set2]] macro-F1 = 0.5245735382058361
[train[hurricane_harvey_2017_lb10_set3]] macro-F1 = 0.5343235437812864
[train[kerala_floods_2018_lb10_set1]] macro-F1 = 0.37585631150317067
[train[kerala_floods_2018_lb10_set2]] macro-F1 = 0.25070607770967424
[train[kerala_floods_2018_lb10_set3]] macro-F1 = 0.31583418042692063
[train[hurricane_dorian_2019_lb10_set1]] macro-F1 = 0.40059934537176667
[train[hurricane_dorian_2019_lb10_set2]] macro-F1 = 0.5296241591966668
[train[hurricane_dorian_2019_lb10_set3]] macro-F1 = 0.4868826589025315
[train[canada_wildfires_2016_lb10_set1]] macro-F1 = 0.5497177820080568
[train[canada_wildfires_2016_lb10_set2]] macro-F1 = 0.53018196960328
[train[canada_wildfires_2016_lb10_set3]] macro-F1 = 0.26345610566465416
[train[hurricane_maria_2017_lb10_set1]] macro-F1 = 0.5632196077405299
[train[hurricane_maria_2017_lb10_set2]] macro-F1 = 0.5039893124856226
[train[hurricane_maria_2017_lb10_set3]] macro-F1 = 0.5319634242169465
[train[kaikoura_earthquake_2016_lb10_set1]] macro-F1 = 0.531335109893306
[train[kaikoura_earthquake_2016_lb10_set2]] macro-F1 = 0.5626861607098825
[train[kaikoura_earthquake_2016_lb10_set3]] macro-F1 = 0.6903194568197574
[train[california_wildfires_2018_lb25_set1]] macro-F1 = 0.5752700382369795
[train[california_wildfires_2018_lb25_set2]] macro-F1 = 0.5998398072920227
[train[california_wildfires_2018_lb25_set3]] macro-F1 = 0.6103413702804964
[train[hurricane_irma_2017_lb25_set1]] macro-F1 = 0.582532130138219
[train[hurricane_irma_2017_lb25_set2]] macro-F1 = 0.6106820054577065
[train[hurricane_irma_2017_lb25_set3]] macro-F1 = 0.6009772100078825
[train[cyclone_idai_2019_lb25_set1]] macro-F1 = 0.5316756347625439
[train[cyclone_idai_2019_lb25_set2]] macro-F1 = 0.5533408806534078
[train[cyclone_idai_2019_lb25_set3]] macro-F1 = 0.5076433020528438
[train[hurricane_florence_2018_lb25_set1]] macro-F1 = 0.6495775308608884
[train[hurricane_florence_2018_lb25_set2]] macro-F1 = 0.6078098610954401
[train[hurricane_florence_2018_lb25_set3]] macro-F1 = 0.5874845376878285
[train[hurricane_harvey_2017_lb25_set1]] macro-F1 = 0.6213098196709798
[train[hurricane_harvey_2017_lb25_set2]] macro-F1 = 0.6105624214463625"""
results_text_to_clipboard_df(text)


✅ Copied to clipboard. Preview:
col                      event       5_1       5_2       5_3      10_1  \
0    california_wildfires_2018  0.090346  0.360013  0.370072  0.513218   
1        canada_wildfires_2016  0.387940  0.260500  0.413789  0.549718   
2            cyclone_idai_2019  0.130752  0.211908  0.258659  0.432312   
3        hurricane_dorian_2019  0.317809  0.325577  0.430853  0.400599   
4      hurricane_florence_2018  0.327223  0.458125  0.397232  0.508538   
5        hurricane_harvey_2017  0.191822  0.139996  0.318668  0.563431   
6          hurricane_irma_2017  0.151363  0.412501  0.221361  0.541695   
7         hurricane_maria_2017  0.271295  0.378790  0.452797  0.563220   
8     kaikoura_earthquake_2016  0.467539  0.220182  0.509041  0.531335   
9           kerala_floods_2018  0.274459  0.343791  0.216566  0.375856   

col      10_2      10_3      25_1      25_2      25_3  50_1  50_2  50_3  
0    0.479358  0.449941  0.575270  0.599840  0.610341   NaN   NaN   NaN  
1    

col,event,5_1,5_2,5_3,10_1,10_2,10_3,25_1,25_2,25_3,50_1,50_2,50_3
0,california_wildfires_2018,0.090346,0.360013,0.370072,0.513218,0.479358,0.449941,0.575270,0.599840,0.610341,NaN,NaN,NaN
1,canada_wildfires_2016,0.387940,0.260500,0.413789,0.549718,0.530182,0.263456,NaN,NaN,NaN,NaN,NaN,NaN
2,cyclone_idai_2019,0.130752,0.211908,0.258659,0.432312,0.305115,0.333275,0.531676,0.553341,0.507643,NaN,NaN,NaN
3,hurricane_dorian_2019,0.317809,0.325577,0.430853,0.400599,0.529624,0.486883,NaN,NaN,NaN,NaN,NaN,NaN
4,hurricane_florence_2018,0.327223,0.458125,0.397232,0.508538,0.590715,0.504607,0.649578,0.607810,0.587485,NaN,NaN,NaN
5,hurricane_harvey_2017,0.191822,0.139996,0.318668,0.563431,0.524574,0.534324,0.621310,0.610562,NaN,NaN,NaN,NaN
6,hurricane_irma_2017,0.151363,0.412501,0.221361,0.541695,0.448069,0.505802,0.582532,0.610682,0.600977,NaN,NaN,NaN
7,hurricane_maria_2017,0.271295,0.378790,0.452797,0.563220,0.503989,0.531963,NaN,NaN,NaN,NaN,NaN,NaN
8,kaikoura_earthquake_2016,0.467539,0.220182,0.509041,0.531335,0.562686,0.690319,NaN,NaN,NaN,NaN,NaN,NaN
9,kerala_floods_2018,0.274459,0.343791,0.216566,0.375856,0.250706,0.315834,NaN,NaN,NaN,NaN,NaN,NaN
